# apache clipper 
## demo 2 : Image and test

# A. 使用 bytes 格式解析圖片


In [1]:
import base64
import json
import requests
from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers import python as python_deployer

引用函示庫

In [2]:
raw_bytes = open('imgs/clipper-logo.png', "rb").read()
raw_bytes[:10]

b'\x89PNG\r\n\x1a\n\x00\x00'

In [3]:
encoded_bytes = base64.b64encode(raw_bytes)
print(type(encoded_bytes))
encoded_bytes[:10]

<class 'bytes'>


b'iVBORw0KGg'

In [4]:
encoded_string = encoded_bytes.decode()
print(type(encoded_string))
encoded_string[:10]

<class 'str'>


'iVBORw0KGg'

## A.1 Build

In [5]:
def image_size(imgs):
    """
    Input: 
    - imgs : (np.ndarray) of shape (n, d). n is the number of data in this batch
             d is the length of the bytes as numpy int8 array.  
    Output:
    - sizes : List[Tuple(int, int),...]
    """
    import base64
    import io
    import os
    import PIL.Image
    import tempfile
  
    num_imgs = len(imgs)
    sizes = []
    for i in range(num_imgs):
        # Create a temp file to write to
        tmp = tempfile.NamedTemporaryFile('wb', delete=False, suffix='.png')
        tmp.write(io.BytesIO(imgs[i]).getvalue())
        tmp.close()
        
        # Use PIL to read in the file and compute size
        size = PIL.Image.open(tmp.name, 'r').size
        
        # Remove the temp file
        os.unlink(tmp.name) 

        sizes.append(size)
    return sizes

In [6]:
from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers import python as python_deployer

clipper_conn = ClipperConnection(DockerContainerManager())

try:
    clipper_conn.start_clipper()
except:
    clipper_conn.connect()


19-09-05:09:51:12 WARNING  [clipper_admin.py:141] [default-cluster] Error starting Clipper: Cluster default-cluster cannot be started because it already exists. Please use ClipperConnection.connect() to connect to it.
19-09-05:09:51:12 INFO     [clipper_admin.py:172] [default-cluster] Successfully connected to Clipper cluster at localhost:1337


## 2.1 & 2.2 & 2.3

In [7]:
python_deployer.deploy_python_closure(clipper_conn, 
                                      name="image-example", 
                                      version=3, 
                                      input_type="bytes", 
                                      func=image_size,
                                      registry="waue0920",
                                      pkgs_to_install=['pillow']
                                     )

19-09-05:09:51:12 INFO     [deployer_utils.py:41] Saving function to /tmp/tmp933iur0fclipper
19-09-05:09:51:12 INFO     [deployer_utils.py:51] Serialized and supplied predict function
19-09-05:09:51:12 INFO     [python.py:192] Python closure saved
19-09-05:09:51:12 INFO     [python.py:206] Using Python 3.6 base image
19-09-05:09:51:12 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmp933iur0fclipper
19-09-05:09:51:13 INFO     [clipper_admin.py:539] [default-cluster] Step 1/3 : FROM clipper/python36-closure-container:0.4.1
19-09-05:09:51:13 INFO     [clipper_admin.py:539] [default-cluster]  ---> e5b9dc250c05
19-09-05:09:51:13 INFO     [clipper_admin.py:539] [default-cluster] Step 2/3 : RUN apt-get -y install build-essential && pip install pillow
19-09-05:09:51:13 INFO     [clipper_admin.py:539] [default-cluster]  ---> Using cache
19-09-05:09:51:13 INFO     [clipper_admin.py:539] [default-cluster]  ---> 8ae0157ca546
19-09-05:09:51:

In [8]:
clipper_conn.register_application(
    name="image-example", 
    input_type="bytes", 
    default_output="-1.0", 
    slo_micros=100000)

19-09-05:09:51:46 INFO     [clipper_admin.py:236] [default-cluster] Application image-example was successfully registered


In [9]:
clipper_conn.link_model_to_app(app_name="image-example", model_name="image-example")


19-09-05:09:51:49 INFO     [clipper_admin.py:303] [default-cluster] Model image-example is now linked to application image-example


## 2.a 
create_endpoint 取代 deploy_python_closure & register_application & link_model_to_app

In [10]:
python_deployer.create_endpoint(
    clipper_conn=clipper_conn, 
    name="image-example2", 
    input_type="bytes", 
    func=image_size, 
    pkgs_to_install=['pillow']
)

19-09-05:09:51:52 INFO     [clipper_admin.py:236] [default-cluster] Application image-example2 was successfully registered
19-09-05:09:51:52 INFO     [deployer_utils.py:41] Saving function to /tmp/tmpbs_ryv62clipper
19-09-05:09:51:52 INFO     [deployer_utils.py:51] Serialized and supplied predict function
19-09-05:09:51:52 INFO     [python.py:192] Python closure saved
19-09-05:09:51:52 INFO     [python.py:206] Using Python 3.6 base image
19-09-05:09:51:52 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmpbs_ryv62clipper
19-09-05:09:51:58 INFO     [clipper_admin.py:539] [default-cluster] Step 1/3 : FROM clipper/python36-closure-container:0.4.1
19-09-05:09:51:58 INFO     [clipper_admin.py:539] [default-cluster]  ---> e5b9dc250c05
19-09-05:09:51:58 INFO     [clipper_admin.py:539] [default-cluster] Step 2/3 : RUN apt-get -y install build-essential && pip install pillow
19-09-05:09:51:58 INFO     [clipper_admin.py:539] [default-cluste

In [11]:
clipper_conn.get_all_apps()

['image-example2', 'hello-world', 'image-example']

## A.2 Test

In [12]:
def query(addr, filename):
    url = "http://%s/image-example2/predict" % addr
    req_json = json.dumps({
        "input":
        base64.b64encode(open(filename, "rb").read()).decode() # bytes to unicode
    })
    headers = {'Content-type': 'application/json'}
    r = requests.post(url, headers=headers, data=req_json)
    print(r.json())

In [26]:
query(clipper_conn.get_query_addr(), 'imgs/clipper-logo.png')

{'query_id': 13, 'output': '(749, 600)', 'default': False}


In [27]:
query(clipper_conn.get_query_addr(), 'imgs/clipper-logo.jpg')

{'query_id': 14, 'output': '(749, 600)', 'default': False}


# B. 換 string 格式

## B.1 Build

In [15]:
 def image_size_json(imgs):
    """
    Input: 
    - imgs : an array of strings 
    Output:
    - sizes : List[Tuple(int, int),...]
    """
    import base64
    import io
    import os
    import PIL.Image
    import tempfile
    import json
  
    num_imgs = len(imgs)
    sizes = []
    for i in range(num_imgs):
        # Deserialize the query
        data = json.loads(imgs[i])
        image_format = data['format']
        image_bytes = data['data'].encode()
        
        # Create a temp file to write to
        tmp = tempfile.NamedTemporaryFile('wb', delete=False, suffix='.{}'.format(image_format))
        tmp.write(io.BytesIO(base64.b64decode(image_bytes)).getvalue())
        tmp.close()
        
        # Use PIL to read in the file and compute size
        size = PIL.Image.open(tmp.name, 'r').size
        
        # Remove the temp file
        os.unlink(tmp.name) 

        sizes.append(size)
    return sizes    
    
    

In [16]:
python_deployer.create_endpoint(
    clipper_conn=clipper_conn, 
    name="image-example-string", 
    registry="waue0920",
    version=2,
    input_type="strings", 
    func=image_size_json, 
    pkgs_to_install=['pillow']
)

19-09-05:09:52:12 INFO     [clipper_admin.py:236] [default-cluster] Application image-example-string was successfully registered
19-09-05:09:52:12 INFO     [deployer_utils.py:41] Saving function to /tmp/tmpm5vz2k9wclipper
19-09-05:09:52:12 INFO     [deployer_utils.py:51] Serialized and supplied predict function
19-09-05:09:52:12 INFO     [python.py:192] Python closure saved
19-09-05:09:52:12 INFO     [python.py:206] Using Python 3.6 base image
19-09-05:09:52:12 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmpm5vz2k9wclipper
19-09-05:09:52:14 INFO     [clipper_admin.py:539] [default-cluster] Step 1/3 : FROM clipper/python36-closure-container:0.4.1
19-09-05:09:52:14 INFO     [clipper_admin.py:539] [default-cluster]  ---> e5b9dc250c05
19-09-05:09:52:14 INFO     [clipper_admin.py:539] [default-cluster] Step 2/3 : RUN apt-get -y install build-essential && pip install pillow
19-09-05:09:52:14 INFO     [clipper_admin.py:539] [default-

In [17]:
clipper_conn.get_all_apps()

['image-example2', 'image-example-string', 'hello-world', 'image-example']

## B.2 test

In [18]:
def query_json(addr, filename, image_format):
    url = "http://%s/image-example-string/predict" % addr
    req_json = json.dumps({
        "input":
        json.dumps({
            'data': base64.b64encode(open(filename, "rb").read()).decode(),
            'format': image_format
        })
    })
    headers = {'Content-type': 'application/json'}
    r = requests.post(url, headers=headers, data=req_json)
    print(r.json())

In [23]:
query_json(clipper_conn.get_query_addr(), 'imgs/clipper-logo.jpg', 'jpg')

{'query_id': 10, 'output': '(749, 600)', 'default': False}


In [24]:
query_json(clipper_conn.get_query_addr(), 'imgs/clipper-logo.png', 'png')

{'query_id': 11, 'output': '(749, 600)', 'default': False}


In [25]:
query_json(clipper_conn.get_query_addr(), 'imgs/encoding_explained.png', 'png')

{'query_id': 12, 'output': '(709, 176)', 'default': False}


# 查目前有幾個微服務

In [22]:
clipper_conn.get_all_apps()

['image-example2', 'image-example-string', 'hello-world', 'image-example']

# Shutdown Clipper

clipper_conn.stop_all()